In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
sns.set()

# You can configure the format of the images: ‘png’, ‘retina’, ‘jpeg’, ‘svg’, ‘pdf’.
%config InlineBackend.figure_format = 'svg'
# this statement allows the visuals to render within your Jupyter Notebook
%matplotlib inline 

In [2]:
df = pd.read_csv('../data/KaggleV2-May-2016.csv')

In [3]:
df

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No


# Data Cleaning & Feature Engineering

### General Checks

In [4]:
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [5]:
df.info() ### check dtypes and for missing values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [6]:
df.AppointmentID.value_counts() ### all appointment ids are unique! 

5769215    1
5731652    1
5707080    1
5702986    1
5715276    1
          ..
5586290    1
5584243    1
5598584    1
5602682    1
5771266    1
Name: AppointmentID, Length: 110527, dtype: int64

## Recoding 

In [7]:
### Cast target and gender feature as binaries
### Yes = 1
### F = 1

df['No-show'] = pd.get_dummies(df['No-show'])
df['No-show'] = pd.get_dummies(df['No-show']) ### repeating because I want Yes to be '1'
df['Gender'] = pd.get_dummies(df['Gender'])

In [8]:
### This feature is suppose to be true/false 

df['Handcap'].replace([2,3,4],1, inplace = True)

In [9]:
### remove negative value for age 
df['Age'].replace(-1,np.NaN, inplace=True)

## Feature Engineering - Scheduled and Appointment Dates 

In [10]:
### Split time stamp from date for date time features 

df['ScheduledDay_Day'] = df['ScheduledDay'].str.split(pat='T', expand = True)[0]
df['AppointmentDay_Day'] = df['AppointmentDay'].str.split(pat='T', expand = True)[0]

df['AppointmentDay_Time'] = df['AppointmentDay'].str.split(pat='T', expand = True)[1]
df['ScheduledDay_Time'] = df['ScheduledDay'].str.split(pat='T', expand = True)[1]

In [11]:
df['AppointmentDay_Time'].value_counts() ### thre are no time stamps for appointment day 

00:00:00Z    110527
Name: AppointmentDay_Time, dtype: int64

In [12]:
### remove the trailing Z

df['ScheduledDay_Time'] = df['ScheduledDay_Time'].str.translate({ord('Z'): None})

In [13]:
### Create feature for hour of the day for scheduled 

df['ScheduledDay_Hours'] = df['ScheduledDay_Time'].apply(lambda x:x[0:2])

In [14]:
### Cast datetime features  

df['ScheduledDay_Day'] = pd.to_datetime(df['ScheduledDay_Day'], infer_datetime_format=True)
df['AppointmentDay_Day'] = pd.to_datetime(df['AppointmentDay_Day'], infer_datetime_format=True)

In [15]:
### Create feature for scheduled date (DD) 

df['ScheduledDay_Date'] = df['ScheduledDay_Day'].astype(str).apply(lambda x:x[-2:])
df['AppointmentDay_Date'] = df['AppointmentDay_Day'].astype(str).apply(lambda x:x[-2:])

In [16]:
### Calculate time lag between ScheduleDay and AppointmentDay

df['day_difference'] = df['AppointmentDay_Day'] - df['ScheduledDay_Day']

In [17]:
df['day_difference'].describe() ### there seems to be negative date differences...

count                     110527
mean     10 days 04:24:31.828602
std      15 days 06:07:11.673762
min            -6 days +00:00:00
25%              0 days 00:00:00
50%              4 days 00:00:00
75%             15 days 00:00:00
max            179 days 00:00:00
Name: day_difference, dtype: object

In [18]:
df['day_difference'] = df['day_difference'].apply(lambda x: x.days) ### extract only the days 
df['day_difference'].astype(int)

0          0
1          0
2          0
3          0
4          0
          ..
110522    35
110523    35
110524    41
110525    41
110526    41
Name: day_difference, Length: 110527, dtype: int32

In [19]:
df[df['day_difference'] < 0] ### there are only 5 entries, guess we can drop them 

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,SMS_received,No-show,ScheduledDay_Day,AppointmentDay_Day,AppointmentDay_Time,ScheduledDay_Time,ScheduledDay_Hours,ScheduledDay_Date,AppointmentDay_Date,day_difference
27033,7.839273e+12,5679978,0,2016-05-10T10:51:53Z,2016-05-09T00:00:00Z,38.0,RESISTÊNCIA,0,0,0,...,0,1,2016-05-10,2016-05-09,00:00:00Z,10:51:53,10,10,09,-1
55226,7.896294e+12,5715660,1,2016-05-18T14:50:41Z,2016-05-17T00:00:00Z,19.0,SANTO ANTÔNIO,0,0,0,...,0,1,2016-05-18,2016-05-17,00:00:00Z,14:50:41,14,18,17,-1
64175,2.425226e+13,5664962,1,2016-05-05T13:43:58Z,2016-05-04T00:00:00Z,22.0,CONSOLAÇÃO,0,0,0,...,0,1,2016-05-05,2016-05-04,00:00:00Z,13:43:58,13,05,04,-1
71533,9.982316e+14,5686628,1,2016-05-11T13:49:20Z,2016-05-05T00:00:00Z,81.0,SANTO ANTÔNIO,0,0,0,...,0,1,2016-05-11,2016-05-05,00:00:00Z,13:49:20,13,11,05,-6
72362,3.787482e+12,5655637,0,2016-05-04T06:50:57Z,2016-05-03T00:00:00Z,7.0,TABUAZEIRO,0,0,0,...,0,1,2016-05-04,2016-05-03,00:00:00Z,06:50:57,06,04,03,-1


In [20]:
df.loc[df['day_difference'] < 0] = np.nan ### replace with nan so that they will be dropped later

In [21]:
### yay we can also check the day of the week for appointments and schedule 

df['appointment_weekday'] = df['AppointmentDay_Day'].apply(lambda x:x.isoweekday())
df['schedule_weekday'] = df['ScheduledDay_Day'].apply(lambda x:x.isoweekday())

In [22]:
### let's more sunday entries, they seem to be special cases 
df['appointment_weekday'].replace(6,np.NaN, inplace=True)
df['schedule_weekday'].replace(6,np.NaN, inplace=True)

In [23]:
df['appointment_weekday'].value_counts() ### some sunday appointments... are these special cases?

3.0    25866
2.0    25638
1.0    22714
5.0    19019
4.0    17246
Name: appointment_weekday, dtype: int64

In [24]:
df['schedule_weekday'].value_counts() ### hmmm, looks like need to make appointment at the clinic 

2.0    26167
3.0    24259
1.0    23085
5.0    18915
4.0    18072
Name: schedule_weekday, dtype: int64

In [25]:
### Let's fix the dtypes 
df.dropna(inplace = True)
df['ScheduledDay_Hours'] = df['ScheduledDay_Hours'].astype(int)
df['ScheduledDay_Date'] = df['ScheduledDay_Date'].astype(int)
df['AppointmentDay_Date'] = df['AppointmentDay_Date'].astype(int)


## More Feature Engineering 

In [26]:
### let's create a new feature to tell us how many appointments 
### we'll be only keep the last entry for the patient, so we need to minus 1
### appointment from the total 

appointments = df.groupby('PatientId').AppointmentID.count().reset_index()
appointments.columns = ['PatientId','prior_appointments']
appointments['prior_appointments'] = appointments['prior_appointments'].apply(lambda x:x-1)
df = df.merge(appointments, how = 'left', on = 'PatientId')

In [27]:
### repeat the step for no-shows 

no_shows = df.groupby('PatientId')['No-show'].sum().reset_index()
no_shows.columns = ['PatientId','prior_no_shows']
no_shows['prior_no_shows'] = no_shows['prior_no_shows'].apply(lambda x:x-1)
df = df.merge(no_shows, how = 'left', on = 'PatientId')

df['prior_no_shows'].replace(-1,0, inplace = True)

In [28]:
### create feature for no. of conditions 
df['total_conditions'] = df['Hipertension']+df['Diabetes']+df['Handcap']+df['Alcoholism']

In [29]:
df['total_conditions'].value_counts()

0.0    84080
1.0    18105
2.0     7650
3.0      618
4.0       13
Name: total_conditions, dtype: int64

In [30]:
df.Neighbourhood.value_counts()

### there are way too many variables to create dummies.
### can't really find additional info about the neigbourhoods to append 

JARDIM CAMBURI                 7717
MARIA ORTIZ                    5805
RESISTÊNCIA                    4429
JARDIM DA PENHA                3877
ITARARÉ                        3514
                               ... 
ILHA DO BOI                      35
ILHA DO FRADE                    10
AEROPORTO                         8
ILHAS OCEÂNICAS DE TRINDADE       2
PARQUE INDUSTRIAL                 1
Name: Neighbourhood, Length: 81, dtype: int64

## Drop duplicate PatientIds, only keep latest appointment entry 

In [31]:
df

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,ScheduledDay_Time,ScheduledDay_Hours,ScheduledDay_Date,AppointmentDay_Date,day_difference,appointment_weekday,schedule_weekday,prior_appointments,prior_no_shows,total_conditions
0,2.987250e+13,5642903.0,1.0,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62.0,JARDIM DA PENHA,0.0,1.0,0.0,...,18:38:08,18,29,29,0.0,5.0,5.0,1,0.0,1.0
1,5.589978e+14,5642503.0,0.0,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56.0,JARDIM DA PENHA,0.0,0.0,0.0,...,16:08:27,16,29,29,0.0,5.0,5.0,1,0.0,0.0
2,4.262962e+12,5642549.0,1.0,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62.0,MATA DA PRAIA,0.0,0.0,0.0,...,16:19:04,16,29,29,0.0,5.0,5.0,1,0.0,0.0
3,8.679512e+11,5642828.0,1.0,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8.0,PONTAL DE CAMBURI,0.0,0.0,0.0,...,17:29:31,17,29,29,0.0,5.0,5.0,1,0.0,0.0
4,8.841186e+12,5642494.0,1.0,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56.0,JARDIM DA PENHA,0.0,1.0,1.0,...,16:07:23,16,29,29,0.0,5.0,5.0,0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110461,2.572134e+12,5651768.0,1.0,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56.0,MARIA ORTIZ,0.0,0.0,0.0,...,09:15:35,9,3,7,35.0,2.0,2.0,1,0.0,0.0
110462,3.596266e+12,5650093.0,1.0,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51.0,MARIA ORTIZ,0.0,0.0,0.0,...,07:27:33,7,3,7,35.0,2.0,2.0,3,0.0,0.0
110463,1.557663e+13,5630692.0,1.0,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21.0,MARIA ORTIZ,0.0,0.0,0.0,...,16:03:52,16,27,7,41.0,2.0,3.0,0,0.0,0.0
110464,9.213493e+13,5630323.0,1.0,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38.0,MARIA ORTIZ,0.0,0.0,0.0,...,15:09:23,15,27,7,41.0,2.0,3.0,1,0.0,0.0


In [32]:
df.sort_values('AppointmentDay', inplace = True) 

In [33]:
df.drop_duplicates(subset='PatientId', keep = 'last', inplace = True)

## Dropping redundant columns and NAs

In [34]:
### dropping redundant featyres 
df = df.drop(['PatientId','ScheduledDay','AppointmentDay','ScheduledDay_Day','AppointmentDay_Day','ScheduledDay_Time','AppointmentID','AppointmentDay_Time','Neighbourhood'], axis =1)

In [35]:
df.dropna(inplace = True) ### removing NA values

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62274 entries, 2149 to 93991
Data columns (total 18 columns):
Gender                 62274 non-null float64
Age                    62274 non-null float64
Scholarship            62274 non-null float64
Hipertension           62274 non-null float64
Diabetes               62274 non-null float64
Alcoholism             62274 non-null float64
Handcap                62274 non-null float64
SMS_received           62274 non-null float64
No-show                62274 non-null float64
ScheduledDay_Hours     62274 non-null int32
ScheduledDay_Date      62274 non-null int32
AppointmentDay_Date    62274 non-null int32
day_difference         62274 non-null float64
appointment_weekday    62274 non-null float64
schedule_weekday       62274 non-null float64
prior_appointments     62274 non-null int64
prior_no_shows         62274 non-null float64
total_conditions       62274 non-null float64
dtypes: float64(14), int32(3), int64(1)
memory usage: 8.3 MB


In [37]:
df.columns = ['gender', 'age', 'scholarship', 'hypertension','diabetes','alcoholism',
              'handicap','sms','no_show','schedule_hour','schedule_day','appointment_day','day_difference',
             'appointment_weekday','schedule_weekday', 'prior_appointments', 'prior_noshows', 'total_conditions']

In [38]:
df['age'] = df['age'].astype(int)

## Saving File 

In [39]:
import pickle 

In [40]:
df.to_pickle('df.pickle')